In [ ]:
#Only run in colab
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Github/Taskmaster
#!pip install tensorflow --upgrade
import tensorflow as tf
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

In [1]:
import rubiks as rubiks
import numpy as np
from student import student
from classroom import classroom
from teacher import teacher
from general_task_network_experimental import student_network
from utils import dotdict
from keras import regularizers

task = rubiks.rubiks_task
setup = rubiks.rubiks_setup
rng = np.random.default_rng(seed=0)

core_params = dotdict({
            "residual_weights_reg" : None,
            "residual_bias_reg" : None,
            "relu_leak" : 0.05,
            "residual_units" : 512,
            "learning_rate" : 0.001,
            "residual_layers" : 1
})

value_network_params = dotdict({
            "residual_weights_reg" : None,
            "residual_bias_reg" : None,
            "relu_leak" : 0.05,
            "residual_units" : 512,
            "learning_rate" : 0.001,
            "post_core_residual_layers" : 2,
            "reward_fork_layers": 1,
            "value_fork_layers": 1
})

state_network_params = dotdict({
            "residual_weights_reg" : None,
            "residual_bias_reg" : None,
            "relu_leak" : 0.05,
            "residual_units" : 300,
            "learning_rate" : 0.001,
            "post_core_residual_layers" : 2
})

params = dotdict({
    "core_params":core_params,
    "value_network_params": value_network_params,
    "state_network_params": state_network_params,
    "state_size":task.input_size,
    "action_codes": task.n_actions
})

my_student_network = student_network.create(params)
#my_student_network = student_network.load("models/combined/trained/trained")
student_template = student(task, 50,my_student_network,0.95)
t = teacher(setup, lambda n : 1+rng.poisson(lam=3,size=n))
c = classroom(task, setup, t, student_template, n_students=1, max_steps=10, buffer_size = lambda n : 2048)

c.max_steps=5
c.buffer_size = lambda n : np.minimum(4096,np.maximum(1024,n//2))
t.step_dist = lambda n : 3*np.ones(n,dtype=int)

In [2]:

for i in range(50):
    print(f"Round {i}.")
    c.run_training_batch(n_problems=100,epochs_per_episode=3)
    #if i % 10 == 9:
    #    my_student_network.save("models/trained.h5")

Round 0.
Before step 1, 98 out of 100 remain open.
Before step 2, 92 out of 100 remain open.
Before step 3, 85 out of 100 remain open.
Before step 4, 83 out of 100 remain open.
Before step 5, 83 out of 100 remain open.
After step 5, 83 out of 100 remain open.


C:\Users\leove\AppData\Roaming\Python\Python310\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/3

7/7 [==============================] - 5s 70ms/step - loss: 0.3652 - value_output_loss: 0.1092 - reward_output_loss: 0.1898 - reward_confidence_output_loss: 0.0662
Epoch 2/3
7/7 [==============================] - 0s 70ms/step - loss: 0.3802 - value_output_loss: 0.0505 - reward_output_loss: 0.2579 - reward_confidence_output_loss: 0.0717
Epoch 3/3
7/7 [==============================] - 0s 67ms/step - loss: 0.3661 - value_output_loss: 0.0506 - reward_output_loss: 0.2479 - reward_confidence_output_loss: 0.0675
Round 1.
Before step 1, 99 out of 100 remain open.
Before step 2, 96 out of 100 remain open.
Before step 3, 96 out of 100 remain open.
Before step 4, 95 out of 100 remain open.
Before step 5, 93 out of 100 remain open.
After step 5, 93 out of 100 remain open.


C:\Users\leove\AppData\Roaming\Python\Python310\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


29/29 [==============================] - 1s 12ms/step - loss: 0.2027 - value_output_loss: 0.0366 - reward_output_loss: 0.1188 - reward_confidence_output_loss: 0.0473
Epoch 1/3
15/15 [==============================] - 1s 60ms/step - loss: 0.2001 - value_output_loss: 0.0366 - reward_output_loss: 0.1218 - reward_confidence_output_loss: 0.0417
Epoch 2/3
15/15 [==============================] - 1s 63ms/step - loss: 0.2042 - value_output_loss: 0.0367 - reward_output_loss: 0.1248 - reward_confidence_output_loss: 0.0428
Epoch 3/3
29/29 [==============================] - 1s 13ms/step
Round 2.
Before step 1, 100 out of 100 remain open.
Before step 2, 96 out of 100 remain open.
Before step 3, 96 out of 100 remain open.
Before step 4, 94 out of 100 remain open.
Before step 5, 94 out of 100 remain open.
After step 5, 92 out of 100 remain open.


...

C:\Users\leove\AppData\Roaming\Python\Python310\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/3
16/16 [==============================] - 1s 75ms/step - loss: 0.0731 - value_output_loss: 0.0675 - reward_output_loss: 0.0028 - reward_confidence_output_loss: 0.0028
Epoch 2/3
16/16 [==============================] - 1s 72ms/step - loss: 0.0731 - value_output_loss: 0.0675 - reward_output_loss: 0.0028 - reward_confidence_output_loss: 0.0028
Epoch 3/3
16/16 [==============================] - 1s 74ms/step - loss: 0.0731 - value_output_loss: 0.0675 - reward_output_loss: 0.0028 - reward_confidence_output_loss: 0.0028
Round 7.
Before step 1, 100 out of 100 remain open.
Before step 2, 86 out of 100 remain open.
Before step 3, 83 out of 100 remain open.
Before step 4, 83 out of 100 remain open.
Before step 5, 81 out of 100 remain open.
After step 5, 80 out of 100 remain open.
Epoch 1/3
16/16 [==============================] - 1s 72ms/step - loss: 0.0726 - value_output_loss: 0.0672 - reward_output_loss: 0.0027 - reward_confidence_output_loss: 0.0027
Epoch 2/3
16/16 [==================

C:\Users\leove\AppData\Roaming\Python\Python310\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/3
16/16 [==============================] - 1s 77ms/step - loss: 0.0857 - value_output_loss: 0.0798 - reward_output_loss: 0.0030 - reward_confidence_output_loss: 0.0030
Epoch 2/3
16/16 [==============================] - 1s 76ms/step - loss: 0.0857 - value_output_loss: 0.0798 - reward_output_loss: 0.0030 - reward_confidence_output_loss: 0.0030
Epoch 3/3
16/16 [==============================] - 1s 83ms/step - loss: 0.0857 - value_output_loss: 0.0798 - reward_output_loss: 0.0030 - reward_confidence_output_loss: 0.0030
Round 11.
Before step 1, 100 out of 100 remain open.
Before step 2, 87 out of 100 remain open.
Before step 3, 84 out of 100 remain open.
Before step 4, 83 out of 100 remain open.
Before step 5, 82 out of 100 remain open.
After step 5, 81 out of 100 remain open.
Epoch 1/3
16/16 [==============================] - 1s 77ms/step - loss: 0.0849 - value_output_loss: 0.0790 - reward_output_loss: 0.0030 - reward_confidence_output_loss: 0.0030
Epoch 2/3
16/16 [=================

KeyboardInterrupt: 

In [7]:
n=8192
rng = np.random.default_rng(seed=0)
states = np.arange(task.state_size)[None,:].repeat(n,axis=0)

for k in range(20):
    print(f"Round {k}.")
    c.run_training_batch(n_problems=100,epochs_per_episode=3)

    for i in range(50):
        _, states = rubiks.task_action(states,rng.choice(a=task.n_actions,size=n))

    actions = rng.choice(a=task.n_actions,size=n)
    _, next_states = rubiks.task_action(states,actions)

    actions = (np.arange(task.n_actions) == actions[...,None]).astype(float)
    colors = rubiks.make_neural_input(states)
    next_colors = rubiks.make_neural_input(next_states)
    my_student_network.fit_state(colors,actions,next_colors,epochs = 5)

    #if k % 10 == 9:
    #  my_student_network.save("models/combined/trained")

Round 0.
Before step 1, 98 out of 100 remain open.
Before step 2, 89 out of 100 remain open.
Before step 3, 89 out of 100 remain open.
Before step 4, 87 out of 100 remain open.
Before step 5, 85 out of 100 remain open.
After step 5, 85 out of 100 remain open.
Epoch 1/3
7/7 [==============================] - 9s 22ms/step - loss: 0.6726 - value_output_loss: 0.3565 - reward_output_loss: 0.1344 - reward_confidence_output_loss: 0.1817
Epoch 2/3
7/7 [==============================] - 0s 24ms/step - loss: 0.5819 - value_output_loss: 0.3048 - reward_output_loss: 0.1163 - reward_confidence_output_loss: 0.1608
Epoch 3/3
7/7 [==============================] - 0s 22ms/step - loss: 0.4587 - value_output_loss: 0.2198 - reward_output_loss: 0.0989 - reward_confidence_output_loss: 0.1400
Epoch 1/5
120/120 [==============================] - 9s 23ms/step - loss: 0.5971 - val_loss: 0.4385
Epoch 2/5
120/120 [==============================] - 2s 18ms/step - loss: 0.4116 - val_loss: 0.4118
Epoch 3/5
120/120 

KeyboardInterrupt: 

In [21]:
states =  np.arange(54,dtype=int)[None,:]
actions = np.array([9],dtype=int)
prediction = my_student_network.predict_state(task.make_input(states),(np.arange(task.n_actions) == actions[...,None]).astype(float))
prediction.reshape(1,54,6).argmax(axis=2), rubiks.start_coloring[rubiks.task_action(states,actions)[1][0]]

1/1 [==============================] - 0s 41ms/step


(array([[0, 2, 4, 0, 2, 0, 2, 5, 4, 1, 4, 4, 1, 0, 3, 4, 0, 3, 0, 3, 5, 2,
         4, 2, 2, 5, 1, 0, 3, 4, 3, 3, 5, 1, 2, 4, 1, 2, 1, 2, 5, 5, 1, 5,
         5, 0, 1, 3, 4, 1, 3, 1, 3, 5]], dtype=int64),
 array([0, 2, 4, 0, 2, 0, 2, 5, 4, 1, 4, 4, 0, 0, 3, 4, 0, 3, 0, 3, 5, 2,
        4, 2, 2, 5, 1, 0, 3, 4, 3, 3, 5, 1, 2, 4, 1, 2, 1, 2, 5, 5, 1, 5,
        5, 0, 1, 3, 4, 1, 3, 1, 3, 5]))